In [91]:
%matplotlib inline

In [108]:
from __future__ import division
from sklearn.datasets import fetch_lfw_people
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import numpy as np

LEARNING_RATE = .1
PCA_COMPONENTS = 80
EPOCHS = 150
IMAGE_DIM = (62, 47) # 2914

people = fetch_lfw_people()

X = people.data
y = people.target

# One-hot the label data
labels = np.zeros(shape=(y.shape[0], y.max() + 1))
for i, x in enumerate(y):
    labels[i][x] = 1


In [109]:
import tensorflow as tf
from math import sqrt

input_size = X[0].size
output_size = len(labels[0])

stddev = lambda a, b: sqrt(2.) * sqrt(2. / (a + b))
weight_init = lambda a, b: tf.Variable(tf.random_normal([a, b], stddev=stddev(a, b)))

t_X = tf.placeholder(tf.float32, [None, input_size])
t_Y = tf.placeholder(tf.float32, [None, output_size])

hidden_layer_1 = weight_init(input_size, 200)
hidden_layer_2 = weight_init(200, 50)
output_layer = weight_init(50, output_size)

(13233, 62, 47)


In [95]:
def model (X, h1_layer, h2_layer, output_layer, alpha=.2):
    h1 = tf.nn.leaky_relu(tf.matmul(X, h1_layer), alpha)
    h2 = tf.nn.leaky_relu(tf.matmul(h1, h2_layer), alpha)
    return tf.matmul(h2, output_layer)

In [96]:
t_model = model(
    t_X, 
    hidden_layer_1,
    hidden_layer_2,
    output_layer
)

cost_f = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=t_model,
        labels=t_Y
    )
)

optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cost_f)
predictor = tf.argmax(t_model, 1)

In [100]:
from sklearn.model_selection import KFold
print X.shape
accuracies = []
for train_index, test_index in KFold(n_splits=10).split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    acc = 0
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(EPOCHS):
            for start, end in zip(range(0, len(X_train), 128), range(128, len(X_train) + 1, 128)):
                sess.run(optimizer, feed_dict = {
                    t_X: X_train[start:end],
                    t_Y: y_train[start:end]
                })
            acc = np.mean(np.argmax(y_test, axis=1) == sess.run(predictor, feed_dict={t_X: X_test}))

    accuracies.append(acc)
    break;
mean_accuracy = np.mean(accuracies)
print mean_accuracy

(13233, 2914)


KeyboardInterrupt: 